# **Curation of the Secretory Metabolic Tasks**

In this notebook the aim is to curate and check the secretory metabolic tasks from https://www.sciencedirect.com/science/article/pii/S1096717623001799?via%3Dihub.

In [1]:
import re
import json
import pickle as pkl
import pandas as pd
import numpy as np

from cobra.core import Reaction, Metabolite, GPR
from cobra.io import read_sbml_model, validate_sbml_model, write_sbml_model

In [3]:
# Reading in model
model = read_sbml_model("../data/Human-GEM.xml.gz")
h1_metabolites = set(m.name for m in model.metabolites)
model

Name,HumanGEM
Memory address,76f926f27040
Number of metabolites,8499
Number of reactions,13085
Number of genes,2897
Number of groups,149
Objective expression,1.0*MAR13082 - 1.0*MAR13082_reverse_11d67
Compartments,"Cytosol, Extracellular, Lysosome, Endoplasmic reticulum, Mitochondria, Peroxisome, Golgi apparatus, Nucleus, Inner mitochondria"


In [1]:
from collections import defaultdict

def extract_metabolites_with_compartments(reaction:str):

    reaction_arrows = ['->', '<->']
    for arrow in reaction_arrows:
        reaction = reaction.replace(arrow, ' ')
    
    matches = re.findall(r'([\w\-\(\)]+)\[(\w+)\]', reaction)
    
    metabolite_dict = defaultdict(list)
    
    for metabolite, compartment in matches:
        if compartment not in metabolite_dict[metabolite]:
            metabolite_dict[metabolite].append(compartment)
    
    return dict(metabolite_dict)


def replace_entrez_with_ensembl(gpr_str:str, mapper:dict) -> str:
    
    def replace_id_with_parentheses(match):
        entrez_id = match.group(1)
        return f"({mapper.get(entrez_id, entrez_id)})"
    
    def replace_id_without_parentheses(match):
        entrez_id = match.group(0)
        return mapper.get(entrez_id, entrez_id)
    
    if not isinstance(gpr_str, str):
        return np.nan
    
    gpr_str = re.sub(r'\((\d+)\)', replace_id_with_parentheses, gpr_str)
    gpr_str = re.sub(r'\b\d+\b', replace_id_without_parentheses, gpr_str)
    
    return gpr_str


def replace_metabolite_ids(template: str, mapper: dict) -> str:

    def replace(match):
        metabolite = match.group(0)
        return f"{mapper.get(metabolite, metabolite)}"

    template = re.sub(r'[\w\-\(\)]+\[\w+\]', replace, template)
    return template

## Curating secretory reactions and metabolites

Resource downloaded from the Supplementary files from the original publication.

In [6]:
sec_tasks = pd.read_excel("../data/1-s2.0-S1096717623001799-mmc2.xlsx", sheet_name="Human.tasks", index_col=0)
sec_tasks

,Reaction.abbreviation,Reaction.Name,Gene.Protein.Reaction.(GPR).association.(Entrez.Gene.IDs),Reaction.catalyzed.by,Components.involved.in.reaction,Template.Formula,Pathway
Task,,,,,,,
Co-translational translocation,TRAP,Formation of Translocon-associated protein com...,(6745) and (6746) and (6747) and (6748),NaN,"SSR1, SSR2, SSR3, SSR4",SSR1[r] + SSR2[r] + SSR3[r] + SSR4[r] -> TRAP[r],Canonical
Co-translational translocation,SEC61C,Formation of SEC61 translocon complex,(29927) and (10952) and (23480),NaN,"SEC61A1, SEC61B, SEC61G",SEC61A1[r] + SEC61B[r] + SEC61G[r] -> SEC61C[r],Canonical
Co-translational translocation,BiP_atp_formation,Binding of atp to BiP,3309,NaN,NaN,BiP[r] + atp[r] -> BiP-atp[r],Canonical
Co-translational translocation,BiP_ATPase,atp hydrolysis in BiP,3309,NaN,NaN,BiP-atp[r] + h2o[r] -> BiP-adp[r] + pi[r],Canonical
Co-translational translocation,BiP_NEF,Nucleotide exchange factor of BiP (SIL1-mediated),(64374) and (10525) and (3309),SIL1 and HYOU1,BiP,BiP-adp[r] + atp[r] -> BiP-atp[r] + adp[r],Canonical
...,...,...,...,...,...,...,...
Vesicle secretion,ARF1_gdp_degradation,ARF1 removal from system,375,NaN,ARF1,ARF1-gdp[c] ->,Canonical
Vesicle secretion,RAB8B_gdp_binding,Binding of RAB8B to GDP,51762,NaN,RAB8B,RAB8B[c] + gdp[c] -> RAB8B-gdp[c],Canonical
Vesicle secretion,RAB8B_activation,Binding of RAB proteins to GTP,(58485) and (7157) and (27095) and (51399) and...,TRAPPC1 and TRAPPC2 and TRAPPC3 and TRAPPC4 an...,NaN,RAB8B-gdp[c] + gtp[c] -> RAB8B-gtp[c] + gdp[c],Canonical


Next, we curated the GPR rules to contain EnsemblIDs instead of EntrezIDs according to the Human1 model format. Also, created new reaction IDs.

In [8]:
df = pd.read_csv("../data/ensembl-to-entrez.tsv", sep='\t').dropna()
entrez_to_ensembl = pd.Series(df['ensemblID'].values, index=df['entrezID'].astype(int).astype(str)).to_dict()

sec_reactions = sec_tasks.copy().reset_index().drop(columns="Task")
sec_reactions = sec_reactions[~sec_reactions["Reaction.Name"].duplicated()]
sec_reactions["Reaction.ID"] = ["SR" + str(i+1).zfill(3) for i, _ in enumerate(sec_reactions["Reaction.Name"])]
sec_reactions["GPR.ensembl"] = [replace_entrez_with_ensembl(gpr, entrez_to_ensembl) for gpr in sec_reactions["Gene.Protein.Reaction.(GPR).association.(Entrez.Gene.IDs)"]]

# IMPORTANT: placeholders "?" and "!" were removed
sec_reactions["Template.Formula"] = [re.sub("[!|?]", "1", template) for template in sec_reactions["Template.Formula"]]
sec_reactions[["Reaction.ID", "Reaction.Name", "Template.Formula", "GPR.ensembl"]]

,Reaction.ID,Reaction.Name,Template.Formula,GPR.ensembl
0,SR001,Formation of Translocon-associated protein com...,SSR1[r] + SSR2[r] + SSR3[r] + SSR4[r] -> TRAP[r],(ENSG00000124783) and (ENSG00000163479) and (E...
1,SR002,Formation of SEC61 translocon complex,SEC61A1[r] + SEC61B[r] + SEC61G[r] -> SEC61C[r],(ENSG00000058262) and (ENSG00000106803) and (E...
2,SR003,Binding of atp to BiP,BiP[r] + atp[r] -> BiP-atp[r],ENSG00000044574
3,SR004,atp hydrolysis in BiP,BiP-atp[r] + h2o[r] -> BiP-adp[r] + pi[r],ENSG00000044574
4,SR005,Nucleotide exchange factor of BiP (SIL1-mediated),BiP-adp[r] + atp[r] -> BiP-atp[r] + adp[r],(ENSG00000120725) and (ENSG00000149428) and (...
...,...,...,...,...
158,SR130,Recruitment of clathrin- AP-containing vesicle...,1 XXX-preSV[g] + 24 ARF1-gtp[c] + 120 CLTA[c] ...,(ENSG00000122705) and (ENSG00000175416) and (E...
159,SR131,Fusion of clathrin- AP-coated vesicles with pl...,XXX-AP-coated[c] + STX19[c] + STX11[c] + STX16...,(ENSG00000122705) and (ENSG00000175416) and (E...
161,SR132,Removal of exocyst complex from system,EXOCYST[c] ->,(ENSG00000182473) and (ENSG00000112685) and (E...
165,SR133,Binding of RAB8B to GDP,RAB8B[c] + gdp[c] -> RAB8B-gdp[c],ENSG00000166128


We then extracted all unique metabolites from the reaction templates and their compartments and created a new data frame.

In [9]:
sec_metabolites = [extract_metabolites_with_compartments(reaction) for reaction in sec_reactions["Template.Formula"].values]
combined_metabolites = {}
for m_dict in sec_metabolites:
    for key, value in m_dict.items():
        if key not in combined_metabolites:
            combined_metabolites[key] = value
        else:
            combined_metabolites[key] = combined_metabolites[key] + value
        combined_metabolites[key] = list(set(combined_metabolites[key]))

metabolites_df = []
for i, (metabolite, compartments) in enumerate(combined_metabolites.items()):
    for c in compartments:
        unique_id = "SM" + str(i+1).zfill(3) + c
        metabolites_df.append([unique_id, metabolite, np.nan, c])

metabolites_df = pd.DataFrame(metabolites_df, columns=["metabolite_id", "metabolite_name", "formula", "compartment"])
metabolites_df["in_h1"] = [met in h1_metabolites for met in metabolites_df["metabolite_name"]]
metabolites_df["name_in_template"] = [name + "[" + c + "]" for name, c in zip(metabolites_df["metabolite_name"], metabolites_df["compartment"])]
metabolites_df

,metabolite_id,metabolite_name,formula,compartment,in_h1,name_in_template
0,SM001r,SSR1,NaN,r,False,SSR1[r]
1,SM002r,SSR2,NaN,r,False,SSR2[r]
2,SM003r,SSR3,NaN,r,False,SSR3[r]
3,SM004r,SSR4,NaN,r,False,SSR4[r]
4,SM005r,TRAP,NaN,r,False,TRAP[r]
...,...,...,...,...,...,...
329,SM310c,STX11,NaN,c,False,STX11[c]
330,SM311c,STX16,NaN,c,False,STX16[c]
331,SM312c,STX1B4111619,NaN,c,False,STX1B4111619[c]
332,SM313c,RAB8B-gdp,NaN,c,False,RAB8B-gdp[c]


Next, we updated the reaction templates with the new metabolite IDs.

In [10]:
metabolite_id_mapper = {name: id for name, id in zip(metabolites_df["name_in_template"], metabolites_df["metabolite_id"])}
sec_reactions["Reaction.Formula"] = [replace_metabolite_ids(template, metabolite_id_mapper) for template in sec_reactions["Template.Formula"]]
sec_reactions = sec_reactions[["Reaction.ID", "Reaction.Name", "Reaction.Formula", "Template.Formula", "GPR.ensembl"]].copy()

sec_reactions

,Reaction.ID,Reaction.Name,Reaction.Formula,Template.Formula,GPR.ensembl
0,SR001,Formation of Translocon-associated protein com...,SM001r + SM002r + SM003r + SM004r -> SM005r,SSR1[r] + SSR2[r] + SSR3[r] + SSR4[r] -> TRAP[r],(ENSG00000124783) and (ENSG00000163479) and (E...
1,SR002,Formation of SEC61 translocon complex,SM006r + SM007r + SM008r -> SM009r,SEC61A1[r] + SEC61B[r] + SEC61G[r] -> SEC61C[r],(ENSG00000058262) and (ENSG00000106803) and (E...
2,SR003,Binding of atp to BiP,SM010r + SM011r -> SM012r,BiP[r] + atp[r] -> BiP-atp[r],ENSG00000044574
3,SR004,atp hydrolysis in BiP,SM012r + SM013r -> SM014r + SM015r,BiP-atp[r] + h2o[r] -> BiP-adp[r] + pi[r],ENSG00000044574
4,SR005,Nucleotide exchange factor of BiP (SIL1-mediated),SM014r + SM011r -> SM012r + SM016r,BiP-adp[r] + atp[r] -> BiP-atp[r] + adp[r],(ENSG00000120725) and (ENSG00000149428) and (...
...,...,...,...,...,...
158,SR130,Recruitment of clathrin- AP-containing vesicle...,1 SM294g + 24 SM226c + 120 SM228c + 120 SM229c...,1 XXX-preSV[g] + 24 ARF1-gtp[c] + 120 CLTA[c] ...,(ENSG00000122705) and (ENSG00000175416) and (E...
159,SR131,Fusion of clathrin- AP-coated vesicles with pl...,SM308c + SM309c + SM310c + SM311c + 44 SM013c ...,XXX-AP-coated[c] + STX19[c] + STX11[c] + STX16...,(ENSG00000122705) and (ENSG00000175416) and (E...
161,SR132,Removal of exocyst complex from system,SM293c ->,EXOCYST[c] ->,(ENSG00000182473) and (ENSG00000112685) and (E...
165,SR133,Binding of RAB8B to GDP,SM314c + SM039c -> SM313c,RAB8B[c] + gdp[c] -> RAB8B-gdp[c],ENSG00000166128


### Adding secretory reactions to Human1

Once we had all necessary data for the metabolites, we created the `Metabolite` objects.

In [11]:
cobra_metabolites = dict()
for i in range(metabolites_df.shape[0]):
    cobra_metabolites[metabolites_df.iloc[i,5]] = Metabolite(
        metabolites_df.iloc[i,0],
        name=metabolites_df.iloc[i,1],
        compartment=metabolites_df.iloc[i,3]
    )

cobra_metabolites["SSR1[r]"]

Metabolite identifier,SM001r
Name,SSR1
Memory address,0x76f8adbe8f70
Formula,None
Compartment,r
In 0 reaction(s),


Next, we added the metabolites into the new model.

In [12]:
sec_model = model.copy()
sec_model.add_metabolites(cobra_metabolites.values())

To add all necessary reactions, we iterated over all rows in the `sec_reactions` to add them one by one to the new `sec_model`.

In [13]:
for i in range(sec_reactions.shape[0]):
    
    try:
        # Place holder for new reactions
        r = sec_model.reactions[0].copy()
        r.id = sec_reactions.iloc[i,0]
        r.name = sec_reactions.iloc[i,1]
        
        if isinstance(sec_reactions.iloc[i,4], str):
            r.gene_reaction_rule = sec_reactions.iloc[i,4]
        else:
            r.gene_reaction_rule = ""
        
        # Adding new reaction to model
        sec_model.add_reactions([r])
        
        # Building reaction stoichiometrics from strings
        r.build_reaction_from_string(sec_reactions.iloc[i,2], rev_arrow="<-", fwd_arrow="->", reversible_arrow="<=>")
    
    except TypeError:
        print(i)
        display(sec_reactions.iloc[i,])
        break

In [14]:
sec_model.slim_optimize()

138.46657955800083

In [ ]:
write_sbml_model(sec_model, "../results/Human-GEM-secretory.xml")

In [497]:
validate_sbml_model("../results/Human-GEM-secretory.xml")

(<Model HumanGEM at 0x7ba39c9a76a0>,
 {'SBML_FATAL': [],
  'SBML_ERROR': [],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': [],
  'COBRA_FATAL': [],
  'COBRA_ERROR': [],
  'COBRA_WARNING': [],
  'COBRA_CHECK': []})

## Creating Secretory Task Metadata

In [16]:
sec_task_metadata = pd.read_csv("../results/task-metadata-secretory.txt", delimiter="\t")
sec_task_metadata["taskID"] = [re.sub("sec_", "S", task_id) for task_id in sec_task_metadata["taskID"]]
sec_task_metadata.to_csv("../results/task-metadata-secretory.txt", sep="\t", index=False)

sec_task_metadata

,taskID,Description,System,Subsystem
0,S1,Co-translational translocation,Translocation,Translocation
1,S2,Post-translational translocation,Translocation,Translocation
2,S3,GPI-anchor,Processing in the ER,PTMs
3,S4,N-linked glycosylation,Processing in the ER,PTMs
4,S5,Disulfide bond formation,Processing in the ER,PTMs
5,S6,N-glycan processing (ER),Processing in the ER,Protein folding
6,S7,Calnexin/calreticulin cycle,Processing in the ER,Protein folding
7,S8,Recognition of misfolded protein,Proteostasis,ERAD
8,S9,Retro-translocation & substrate ubiquitination,Proteostasis,ERAD
9,S10,Proteasomal degradation,Proteostasis,ERAD


Now we will create a task structure objects in the `.json` and `.pkl` format.

In [18]:
sec_task_structure = dict()


# JSON
sec_rxn_gprs = {reaction.id: reaction.gpr.to_string() for reaction in sec_model.reactions}

for i in range(sec_task_metadata.shape[0]):
    sec_id = sec_task_metadata.iloc[i,0]
    sec_description = sec_task_metadata.iloc[i,1]
    reactions_names = sec_tasks[sec_tasks.index == sec_description]["Reaction.Name"].values
    reaction_ids = sec_reactions[sec_reactions["Reaction.Name"].isin(reactions_names)]["Reaction.ID"].values.tolist()

    reaction_structure = dict()
    for rxn in reaction_ids:
        reaction_structure[rxn] = sec_rxn_gprs[rxn]

    sec_task_structure[sec_id] = reaction_structure

with open("results/task-structure-secretory.json", "w") as f:
    json.dump(sec_task_structure, f)
    del f


# PICKLE
sec_rxn_gprs = {reaction.id: reaction.gpr for reaction in sec_model.reactions}

for i in range(sec_task_metadata.shape[0]):
    sec_id = sec_task_metadata.iloc[i,0]
    sec_description = sec_task_metadata.iloc[i,1]
    reactions_names = sec_tasks[sec_tasks.index == sec_description]["Reaction.Name"].values
    reaction_ids = sec_reactions[sec_reactions["Reaction.Name"].isin(reactions_names)]["Reaction.ID"].values.tolist()

    reaction_structure = dict()
    for rxn in reaction_ids:
        reaction_structure[rxn] = sec_rxn_gprs[rxn]

    sec_task_structure[sec_id] = reaction_structure

with open("results/task-structure-secretory.pkl", "wb") as f:
    pkl.dump(sec_task_structure, f)
    del f

Finally, we will create a task structure object in the format of a logical matrix, where columns are tasks, and rows are reactions. Each cell contains ones or zeros, indicating whether a reactions participates in a given task.

In [30]:
tasks = sec_task_structure.keys()
reactions = np.unique(np.concat([list(r) for r in sec_task_structure.values()]))
task_structure_matrix = np.zeros(shape=(len(reactions), len(tasks)))

for i, r in enumerate(reactions):
    for j, t in enumerate(tasks):
        if r in sec_task_structure[t]:
            task_structure_matrix[i,j] = 1

pd.DataFrame(task_structure_matrix.astype(int), index=reactions, columns=tasks)\
    .to_csv("../results/task-structure-secretory-matrix.tsv", sep="\t")